In [84]:
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings(action='ignore')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [85]:
rankList = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "EMERALD", "DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]
modelNameList = ["CatBoost", "ExtraTree", "LightGBM", "RandomForest"]
colList = ["accuracy_score", "F1_score", "roc_auc_score"]
idxList = [
    ["IRON", "IRON", "IRON", "IRON", 
    "BRONZE", "BRONZE", "BRONZE", "BRONZE", 
    "SILVER", "SILVER", "SILVER", "SILVER", 
    "GOLD", "GOLD", "GOLD", "GOLD", 
    "PLATINUM", "PLATINUM", "PLATINUM", "PLATINUM", 
    "EMERALD", "EMERALD", "EMERALD", "EMERALD", 
    "DIAMOND", "DIAMOND", "DIAMOND", "DIAMOND", 
    "MASTER", "MASTER", "MASTER", "MASTER", 
    "GRANDMASTER", "GRANDMASTER", "GRANDMASTER", "GRANDMASTER", 
    "CHALLENGER", "CHALLENGER", "CHALLENGER", "CHALLENGER"],
    ["CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest",
     "CatBoost", "ExtraTree", "LightGBM", "RandomForest"]
    ]
scoreList = np.empty((0,3))

for rank in rankList:
    df = pd.read_csv(f'../Dataset/perMinuteDataset/10min/{rank}.csv')

    win_df = df[['Diff_FirstBLOOD', 'Diff_FirstDRAGON',
        'dragonType', 'WIN_controlWARDPlaced', 
        'WIN_Kill_top', 'WIN_Kill_jgl', 'WIN_Kill_mid', 'WIN_Kill_ad', 'WIN_Kill_sup',
        'WIN_Death_top', 'WIN_Death_jgl', 'WIN_Death_mid', 'WIN_Death_ad', 'WIN_Death_sup', 
        'WIN_Asisst_top', 'WIN_Asisst_jgl', 'WIN_Asisst_mid', 'WIN_Asisst_ad', 'WIN_Asisst_sup',
        'WIN_LV_top', 'WIN_LV_jgl', 'WIN_LV_mid', 'WIN_LV_ad', 'WIN_LV_sup',
        'WIN_CS_top', 'WIN_CS_jgl', 'WIN_CS_mid', 'WIN_CS_ad', 'WIN_CS_sup',
        'WIN_jglCS_jgl',
        'WIN_GOLD_top', 'WIN_GOLD_jgl', 'WIN_GOLD_mid', 'WIN_GOLD_ad', 'WIN_GOLD_sup',
        'WIN_WARDkill']]
    lose_df = df[['Diff_FirstBLOOD', 'Diff_FirstDRAGON',
        'dragonType', 'LOSE_controlWARDPlaced',
        'LOSE_Kill_top', 'LOSE_Kill_jgl', 'LOSE_Kill_mid', 'LOSE_Kill_ad', 'LOSE_Kill_sup',
        'LOSE_Death_top', 'LOSE_Death_jgl', 'LOSE_Death_mid', 'LOSE_Death_ad', 'LOSE_Death_sup',
        'LOSE_Asisst_top', 'LOSE_Asisst_jgl', 'LOSE_Asisst_mid', 'LOSE_Asisst_ad', 'LOSE_Asisst_sup',
        'LOSE_LV_top', 'LOSE_LV_jgl', 'LOSE_LV_mid', 'LOSE_LV_ad', 'LOSE_LV_sup',
        'LOSE_CS_top', 'LOSE_CS_jgl', 'LOSE_CS_mid', 'LOSE_CS_ad', 'LOSE_CS_sup',
        'LOSE_jglCS_jgl',
        'LOSE_GOLD_top', 'LOSE_GOLD_jgl', 'LOSE_GOLD_mid', 'LOSE_GOLD_ad', 'LOSE_GOLD_sup',
        'LOSE_WARDkill']]
    colName = 'WIN'
    
    win_df = win_df.rename(columns={f'{colName}_controlWARDPlaced': 'controlWARDPlaced',
                                                    f'{colName}_Kill_top': 'Kill_top',f'{colName}_Kill_jgl': 'Kill_jgl',f'{colName}_Kill_mid': 'Kill_mid',f'{colName}_Kill_ad': 'Kill_ad', f'{colName}_Kill_sup': 'Kill_sup',
                                                    f'{colName}_Death_top': 'Death_top',f'{colName}_Death_jgl': 'Death_jgl',f'{colName}_Death_mid': 'Death_mid',f'{colName}_Death_ad': 'Death_ad',f'{colName}_Death_sup': 'Death_sup',
                                                    f'{colName}_Asisst_top': 'Asisst_top',f'{colName}_Asisst_jgl': 'Asisst_jgl',f'{colName}_Asisst_mid': 'Asisst_mid',f'{colName}_Asisst_ad': 'Asisst_ad',f'{colName}_Asisst_sup': 'Asisst_sup',
                                                    f'{colName}_LV_top': 'LV_top',f'{colName}_LV_jgl': 'LV_jgl',f'{colName}_LV_mid': 'LV_mid',f'{colName}_LV_ad': 'LV_ad',f'{colName}_LV_sup': 'LV_sup',
                                                    f'{colName}_CS_top': 'CS_top',f'{colName}_CS_jgl': 'CS_jgl',f'{colName}_CS_mid': 'CS_mid',f'{colName}_CS_ad': 'CS_ad',f'{colName}_CS_sup': 'CS_sup',
                                                    f'{colName}_jglCS_jgl': 'jglCS_jgl',
                                                    f'{colName}_GOLD_top': 'GOLD_top',f'{colName}_GOLD_jgl': 'GOLD_jgl',f'{colName}_GOLD_mid': 'GOLD_mid',f'{colName}_GOLD_ad': 'GOLD_ad',f'{colName}_GOLD_sup': 'GOLD_sup',
                                                    f'{colName}_WARDkill': 'WARDkill'})

    colName = 'LOSE'
    lose_df = lose_df.rename(columns={f'{colName}_controlWARDPlaced': 'controlWARDPlaced',
                                                    f'{colName}_Kill_top': 'Kill_top',f'{colName}_Kill_jgl': 'Kill_jgl',f'{colName}_Kill_mid': 'Kill_mid',f'{colName}_Kill_ad': 'Kill_ad', f'{colName}_Kill_sup': 'Kill_sup',
                                                    f'{colName}_Death_top': 'Death_top',f'{colName}_Death_jgl': 'Death_jgl',f'{colName}_Death_mid': 'Death_mid',f'{colName}_Death_ad': 'Death_ad',f'{colName}_Death_sup': 'Death_sup',
                                                    f'{colName}_Asisst_top': 'Asisst_top',f'{colName}_Asisst_jgl': 'Asisst_jgl',f'{colName}_Asisst_mid': 'Asisst_mid',f'{colName}_Asisst_ad': 'Asisst_ad',f'{colName}_Asisst_sup': 'Asisst_sup',
                                                    f'{colName}_LV_top': 'LV_top',f'{colName}_LV_jgl': 'LV_jgl',f'{colName}_LV_mid': 'LV_mid',f'{colName}_LV_ad': 'LV_ad',f'{colName}_LV_sup': 'LV_sup',
                                                    f'{colName}_CS_top': 'CS_top',f'{colName}_CS_jgl': 'CS_jgl',f'{colName}_CS_mid': 'CS_mid',f'{colName}_CS_ad': 'CS_ad',f'{colName}_CS_sup': 'CS_sup',
                                                    f'{colName}_jglCS_jgl': 'jglCS_jgl',
                                                    f'{colName}_GOLD_top': 'GOLD_top',f'{colName}_GOLD_jgl': 'GOLD_jgl',f'{colName}_GOLD_mid': 'GOLD_mid',f'{colName}_GOLD_ad': 'GOLD_ad',f'{colName}_GOLD_sup': 'GOLD_sup',
                                                    f'{colName}_WARDkill': 'WARDkill'})

    def mergeCol(colName):
        global win_df, lose_df
        win_df[colName] = (win_df[f'{colName}_ad']+win_df[f'{colName}_top']+win_df[f'{colName}_jgl']+win_df[f'{colName}_mid']+win_df[f'{colName}_sup'])
        lose_df[colName] = (lose_df[f'{colName}_ad']+lose_df[f'{colName}_top']+lose_df[f'{colName}_jgl']+lose_df[f'{colName}_mid']+lose_df[f'{colName}_sup'])
        win_df= win_df.drop([f'{colName}_ad', f'{colName}_top', f'{colName}_jgl', f'{colName}_mid', f'{colName}_sup'],axis=1)
        lose_df= lose_df.drop([f'{colName}_ad', f'{colName}_top', f'{colName}_jgl', f'{colName}_mid', f'{colName}_sup'],axis=1)
        return win_df, lose_df

    win_df, lose_df = mergeCol('Kill')
    win_df, lose_df = mergeCol('Death')
    win_df, lose_df = mergeCol('Asisst')
    win_df, lose_df = mergeCol('LV')
    win_df, lose_df = mergeCol('CS')
    win_df, lose_df = mergeCol('GOLD')

    win_df['Kill'] = win_df['Kill'] - lose_df['Kill'] 
    lose_df['Kill'] = win_df['Kill']*-1
    win_df['Death'] = win_df['Death'] - lose_df['Death'] 
    lose_df['Death'] = win_df['Death']*-1
    win_df['LV'] = win_df['LV']-lose_df['LV']
    lose_df['LV'] = win_df['LV']*-1
    win_df['CS'] = win_df['CS'] - lose_df['CS'] 
    lose_df['CS'] = win_df['CS']*-1
    win_df['GOLD'] = win_df['GOLD'] - lose_df['GOLD'] 
    lose_df['GOLD'] = win_df['GOLD']*-1
    lose_df['Diff_FirstBLOOD'] = win_df['Diff_FirstBLOOD']*-1
    lose_df['Diff_FirstDRAGON'] = win_df['Diff_FirstDRAGON']*-1

    for i in win_df[win_df['Diff_FirstDRAGON'] == -1].index:
        win_df.loc[i, 'dragonType'] = win_df.loc[i, 'dragonType'] * -1
    for i in lose_df[lose_df['Diff_FirstDRAGON'] == -1].index:
        lose_df.loc[i, 'dragonType'] = lose_df.loc[i, 'dragonType'] * -1

    # 이상치 제거
    def remove_outlier(input_data):
        q1 = input_data.quantile(0.25) # 제 1사분위수
        q3 = input_data.quantile(0.75) # 제 3사분위수
        iqr = q3 - q1 # IQR(Interquartile range) 계산
        minimum = q1 - (iqr * 1.5) # IQR 최솟값
        maximum = q3 + (iqr * 1.5) # IQR 최댓값
        # IQR 범위 내에 있는 데이터만 산출(IQR 범위 밖의 데이터는 이상치)
        df_removed_outlier = input_data[(minimum < input_data) & (input_data < maximum)]
        return df_removed_outlier

    win_prep = remove_outlier(win_df)
    win_prep.dropna(axis = 0, how = 'any', inplace = True)
    lose_prep = remove_outlier(lose_df)
    lose_prep.dropna(axis = 0, how = 'any', inplace = True)
    win_prep['result'] = 1
    lose_prep['result'] = 0
    data = pd.concat([win_prep, lose_prep], axis=0)
    data.result.value_counts(normalize=True)

    print('Total data size of ' + rank + ' =', data.shape[0])

    # 랜덤 포레스트
    rf = RandomForestClassifier(max_features='sqrt', max_leaf_nodes=100)
    # 라이트지비엠
    lgbm = LGBMClassifier(n_estimators=100, max_depth=12, num_leaves=25, verbosity=0, min_child_samples=30)
    # 캣부스트
    cat = CatBoostClassifier(iterations=200, depth=5, learning_rate=0.2, l2_leaf_reg=50, verbose=0)
    # 엑스트라트리
    et = ExtraTreesClassifier(max_depth=7, max_features=None)

    x = data[data.columns.difference(['result'])]
    y = data['result']

    modelList = [cat, et, lgbm, rf]
    for i in range(4):
        scores = []
        model = modelList[i]

        x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size = 0.2, random_state = 42)
        y_train = y_train.values.ravel()
        model.fit(x_train, y_train)
        pre_test = model.predict(x_test)
        pre_train = model.predict(x_train)
        if model == cat:
            printModel = f"CatBoost{model.get_params()}"
        else:
            printModel = model
        
        scores.append(str(round(accuracy_score(y_test, pre_test)*100, 2)) + "%")
        scores.append(str(round(f1_score(y_test, pre_test)*100, 2)) + "%")
        scores.append(str(round(roc_auc_score(y_test, pre_test)*100, 2)) + "%")
        scoreList = np.append(scoreList, np.array([scores]), axis=0)

df = pd.DataFrame(scoreList, columns=colList, index=idxList)
print(df)

Total data size of IRON = 11300
Total data size of BRONZE = 10590
Total data size of SILVER = 12386
Total data size of GOLD = 6490
Total data size of PLATINUM = 13398
Total data size of EMERALD = 14693
Total data size of DIAMOND = 13567
Total data size of MASTER = 13597
Total data size of GRANDMASTER = 9537
Total data size of CHALLENGER = 8097
[['78.14%' '78.43%' '78.15%']
 ['76.11%' '76.76%' '76.08%']
 ['78.72%' '78.99%' '78.73%']
 ['78.14%' '78.61%' '78.13%']
 ['74.74%' '75.42%' '74.72%']
 ['74.93%' '75.34%' '74.92%']
 ['73.61%' '74.44%' '73.58%']
 ['74.74%' '75.13%' '74.73%']
 ['79.38%' '79.83%' '79.36%']
 ['77.4%' '77.97%' '77.38%']
 ['80.06%' '80.43%' '80.05%']
 ['78.81%' '79.29%' '78.8%']
 ['76.27%' '75.2%' '76.21%']
 ['76.04%' '74.19%' '75.89%']
 ['74.96%' '73.6%' '74.87%']
 ['76.58%' '75.16%' '76.48%']
 ['77.35%' '78.99%' '77.17%']
 ['78.13%' '80.07%' '77.83%']
 ['77.84%' '79.46%' '77.65%']
 ['78.1%' '79.88%' '77.85%']
 ['77.48%' '77.42%' '77.48%']
 ['77.07%' '77.03%' '77.07%']